In [4]:
import numpy as np
from scipy.optimize import minimize, NonlinearConstraint

# Given data
R = np.array([0.8, 1.16, 1.57])
V = np.array([0.0542, 0.0563, 0.0563])
M1x = 451
Nx = np.array([337, 167, 186, 212])
A0 = M1x * Nx[1:] / Nx.sum()
V_min = np.min(V)


# Assuming arbitrary values for z and p for illustration
z = 1.0
p = 0.3813


# Objective function adjustments
def a0(B_plus, A_plus, z, p):
    # Prevent division by zero by ensuring the denominator is never zero
    return max(1 / (z * (1 - p)) * B_plus - A_plus, 1e-8)

def b0(B_plus, p):
    # Similarly, ensure b0 is never zero to avoid division by zero
    return max(p / (1 - p) * B_plus, 1e-8)

# Assuming the definitions of a0 and b0 functions, z, p, and V are already provided above
def print_initialization(x0, V, z, p):
    a0_value = a0(x0[0], x0[1], z, p)
    b0_value = b0(x0[0], p)
    
    # Compute the denominators
    v_denominators = V - 1 / a0_value - 1 / b0_value
    
    # Print the initialization values
    print("Initialization:")
    print(f"a0: {a0_value}")
    print(f"b0: {b0_value}")
    print(f"v_denominators: {v_denominators}")


def find_feasible_point(V, z, p):
    B_plus = 100  # Initial positive B+
    A_plus = 100   # Start with a small A+
    
    while True:
        a0_value = a0(B_plus, A_plus, z, p)
        b0_value = b0(B_plus, p)
        v_denominators = V - 1 / a0_value - 1 / b0_value
        
        # Check if all conditions are satisfied
        if a0_value > 0 and b0_value > 0 and np.all(v_denominators > 0):
            print(f"Feasible point found: B_+ = {B_plus}, A_+ = {A_plus}")
            print(f"a0: {a0_value}, b0: {b0_value}, v_denominators: {v_denominators}")
            return np.array([B_plus, A_plus])  # Return the feasible point
        else:
            # Adjust B_+ and A_+ if conditions are not satisfied
            B_plus *= 1.1  # Increment B_+ to increase a0 and b0
            A_plus *= 0.9  # Adjust A_+ to ensure a0 > 0, may need fine-tuning


def objective(x, V, R, z, p):
    B_plus, A_plus = x
    a0_value = a0(B_plus, A_plus, z, p)
    b0_value = b0(B_plus, p)
    
    # Ensure denominators are always positive
    v_denominators = V  - 1 / a0_value - 1 / b0_value
    #if np.any(v_denominators <= 0):
        # Handle the case where v_denominators are not positive
        # This is a simple handling approach; you might want to use a different strategy
     #   return np.inf

    term1 = (B_plus - np.sum((1 + b0_value / (a0_value * R)) / v_denominators))**2
    term2 = (A_plus - np.sum((1 + a0_value * R  / b0_value ) / v_denominators))**2
    
    # Print the computed a0, b0, and denominators for inspection
    #print("a0:", a0_value)
    #print("b0:", b0_value)
    #print("v's across the sum:", v_denominators)

    return term1 + term2    
    
    
constraints = [
    # Existing constraints
    {'type': 'ineq', 'fun': lambda x: 1/(z * (1 - p)) * x[0] - x[1]-1}, # Ensuring a0 >= 1, 
    {'type': 'ineq', 'fun': lambda x: p/(1-p)*x[0] - 1},  # Ensuring b0 >= 1, B_+ >= 1 
    {'type': 'ineq', 'fun': lambda x: x[1] -1}  # ensuring A_+ >= 1
]

# Define the nonlinear constraint function
def nonlinear_constraint_func(x, V_min, z, p):
    A_plus = x[1]
    return A_plus**2 + (2 * (z * p - 1) / (z * p * V_min)) * A_plus + ((z * p + 1)**2) / ((z * p * V_min)**2)


#nonlinear_constraint = NonlinearConstraint(lambda x: nonlinear_constraint_func(x, V_min, z, p), -np.inf, 0)
# Add the nonlinear constraint to the existing constraints for the optimization
#constraints.append(nonlinear_constraint)


# Assuming V, z, and p are already defined
x_feas = find_feasible_point(V, z, p)

# You can now use x_feas as your initial guess in optimization
print("Feasible initial guess for optimization:", x_feas)

x_0 = np.array([2,2])
# Optimization
result = minimize(fun=objective, x0=x_feas, args=(V, R, z, p), method='trust-constr', constraints=constraints)
#result = minimize(fun=objective, x0=x_feas, args=(V, R, z, p))

print("Optimization Result:")
print(result)

Feasible point found: B_+ = 100, A_+ = 100
a0: 61.62922256343947, b0: 61.62922256343947, v_denominators: [0.02174786 0.02384786 0.02384786]
Feasible initial guess for optimization: [100 100]
Optimization Result:
           message: `xtol` termination condition is satisfied.
           success: True
            status: 2
               fun: 2.15770081575392e-12
                 x: [ 1.684e+02  1.759e+02]
               nit: 125
              nfev: 357
              njev: 119
              nhev: 0
          cg_niter: 173
      cg_stop_cond: 2
              grad: [ 1.533e-05  1.128e-05]
   lagrangian_grad: [ 1.533e-05  1.128e-05]
            constr: [array([ 9.525e+01]), array([ 1.028e+02]), array([ 1.749e+02])]
               jac: [array([[ 1.616e+00, -1.000e+00]]), array([[ 6.163e-01,  0.000e+00]]), array([[ 0.000e+00,  1.000e+00]])]
       constr_nfev: [357, 357, 357]
       constr_njev: [0, 0, 0]
       constr_nhev: [0, 0, 0]
                 v: [array([-1.509e-09]), array([-9.141e-10

In [5]:
# Assuming result.x contains the optimized values for B_+ and A_+
B_plus_optimized, A_plus_optimized = result.x

# Calculate a0 and b0 using the optimized values of B_+ and A_+
a0_optimized = a0(B_plus_optimized, A_plus_optimized, z, p)
b0_optimized = b0(B_plus_optimized, p)

# Calculate the sums
a0_b0_sum = a0_optimized + b0_optimized
A_B_sum = A_plus_optimized + B_plus_optimized

# Print the results
print(f"a0 (optimized): {a0_optimized}")
print(f"b0 (optimized): {b0_optimized}")
print(f"a0 + b0: {a0_b0_sum}")
print(f"A+ (optimized): {A_plus_optimized}")
print(f"B+ (optimized): {B_plus_optimized}")
print(f"A+ + B+: {A_B_sum}")

a0 (optimized): 96.2518705721165
b0 (optimized): 103.77602542941074
a0 + b0: 200.02789600152724
A+ (optimized): 175.91184678799556
B+ (optimized): 168.38769193070135
A+ + B+: 344.2995387186969


In [6]:
# Assuming a0_value, b0_value, R, and v_denominators are already calculated

#    term2 = (A_plus - np.sum((1 + a0_value * R  / b0_value ) / v_denominators))**2
#    term1 = (B_plus - np.sum((1 + b0_value / (a0_value * R)) / v_denominators))**2



# Calculate A and B components
v_denominators = V - 1 / a0_optimized - 1 / b0_optimized
A_components = (1 + R * a0_optimized / b0_optimized ) / v_denominators
B_components = (1 + b0_optimized / (a0_optimized * R)) / v_denominators
N_components = A_components + B_components

# Print the results for each component
print("Components of A:")
for i, A_comp in enumerate(A_components, 1):
    print(f"A[{i}]: {A_comp}")

print("\nComponents of B:")
for i, B_comp in enumerate(B_components, 1):
    print(f"B[{i}]: {B_comp}")
    

print("\nComponents of N:")
print(f"a0 + b0: {a0_b0_sum}")
for i, N_comp in enumerate(N_components, 1):
    print(f"N[{i}]: {N_comp}")

print("Nx = np.array([337, 167, 186, 212]")

Components of A:
A[1]: 50.97365840627183
A[2]: 57.22747964377605
A[3]: 67.71070957615608

Components of B:
B[1]: 68.69793334348705
B[2]: 53.19055052871652
B[3]: 46.499209264776134

Components of N:
a0 + b0: 200.02789600152724
N[1]: 119.67159174975887
N[2]: 110.41803017249256
N[3]: 114.2099188409322
Nx = np.array([337, 167, 186, 212]
